In [ ]:
```xml
<VSCode.Cell language="markdown">
# Bestandsvergleich: VDEH vs UB TUBAF

Dieses Notebook führt den ersten Vergleich zwischen den VDEH-Neuerwerbungen und dem UB TUBAF-Bestand durch.

## 🎯 Ziel
- Lade beide Datenquellen (VDEH + UB TUBAF)
- Führe Multi-Strategie Matching durch
- Analysiere Überschneidungen und Lücken

## 📊 Datenquellen
- **VDEH**: Neuerwerbungen (~58k Records, OAI-PMH XML)
- **UB TUBAF**: Bibliotheksbestand (MAB2-Format)
</VSCode.Cell>
<VSCode.Cell language="python">
# 🛠️ SETUP: Konfiguration und Module laden
import sys
from pathlib import Path

# Projektroot finden
current_dir = Path.cwd()
project_root = None

for parent in [current_dir] + list(current_dir.parents):
    if (parent / 'config.yaml').exists():
        project_root = parent
        break

if project_root is None:
    raise FileNotFoundError("config.yaml nicht gefunden!")

print(f"📁 Projektroot: {project_root}")

# Python-Pfad erweitern
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
</VSCode.Cell>
<VSCode.Cell language="python">
# 📋 KONFIGURATION LADEN
from config_loader import load_config

config = load_config(project_root / 'config.yaml')
config.print_summary()

print("\n✅ Konfiguration für Dual-Source Vergleich geladen")
</VSCode.Cell>
<VSCode.Cell language="python">
# 📦 BIBLIOTHEKEN UND PARSER LADEN
import pandas as pd
import matplotlib.pyplot as plt
import warnings

# Parser importieren
from parsers.vdeh_parser import parse_bibliography, analyze_bibliography_data
from parsers.mab2_parser import MAB2Parser, analyze_mab2_data

# Comparison Engine importieren
from comparison.matcher import BookMatcher

print("✅ Alle Module geladen")
print("🔍 BookMatcher bereit für Bestandsvergleich")
</VSCode.Cell>
<VSCode.Cell language="python">
# 📚 VDEH DATEN LADEN
print("📚 === VDEH DATEN LADEN ===")

vdeh_file = config.get_path('data_sources.vdeh.path')
max_vdeh_records = 1000  # Test mit Sample

print(f"📂 VDEH-Datei: {vdeh_file}")
print(f"🧪 Lade Sample: {max_vdeh_records:,} Records")

df_vdeh = parse_bibliography(str(vdeh_file), max_records=max_vdeh_records)

print(f"✅ VDEH geladen: {len(df_vdeh):,} Records")
print(f"📋 Spalten: {list(df_vdeh.columns)}")

# Kurze Analyse
analyze_bibliography_data(df_vdeh)
</VSCode.Cell>
<VSCode.Cell language="python">
# 🏛️ UB TUBAF DATEN LADEN  
print("\n🏛️ === UB TUBAF DATEN LADEN ===")

ub_file = config.get_path('data_sources.ub_tubaf.path')
ub_encoding = config.get('data_sources.ub_tubaf.encoding')
max_ub_records = 1000  # Test mit Sample

print(f"📂 UB-Datei: {ub_file}")
print(f"🔤 Encoding: {ub_encoding}")
print(f"🧪 Lade Sample: {max_ub_records:,} Records")

parser_ub = MAB2Parser(encoding=ub_encoding)
df_ub = parser_ub.parse_file(ub_file, max_records=max_ub_records)

print(f"✅ UB TUBAF geladen: {len(df_ub):,} Records")
print(f"📋 Spalten: {list(df_ub.columns)}")

# Kurze Analyse  
analyze_mab2_data(df_ub)
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 🔍 Bestandsvergleich durchführen

Verwendet die konfigurierten Matching-Strategien für den Vergleich.
</VSCode.Cell>
<VSCode.Cell language="python">
# 🔍 BESTANDSVERGLEICH INITIALISIEREN
print("🔍 === BESTANDSVERGLEICH SETUP ===")

# BookMatcher mit Konfiguration initialisieren
matcher = BookMatcher(config.config_dict)

# Zeige konfigurierte Strategien
comparison_config = config.get('comparison', {})
strategies = comparison_config.get('matching_strategies', [])
thresholds = comparison_config.get('similarity_thresholds', {})

print(f"🎯 Matching-Strategien: {strategies}")
print(f"📊 Schwellenwerte: {thresholds}")

print(f"\n📚 Bereit für Vergleich:")
print(f"   VDEH Records: {len(df_vdeh):,}")
print(f"   UB Records: {len(df_ub):,}")
</VSCode.Cell>
<VSCode.Cell language="python">
# 🚀 BESTANDSVERGLEICH DURCHFÜHREN
print("🚀 === BESTANDSVERGLEICH GESTARTET ===")

# Führe vollständigen Vergleich durch
matches_df = matcher.compare_collections(df_vdeh, df_ub)

print(f"\n✅ === VERGLEICH ABGESCHLOSSEN ===")

if not matches_df.empty:
    print(f"📊 Gefundene Matches: {len(matches_df):,}")
    print(f"📈 Match-Rate: {len(matches_df)/len(df_vdeh)*100:.1f}% der VDEH Records")
    
    # Match-Breakdown nach Strategie
    match_breakdown = matches_df['match_type'].value_counts()
    print(f"\n🎯 Matches nach Strategie:")
    for strategy, count in match_breakdown.items():
        print(f"   {strategy}: {count:,}")
else:
    print("❌ Keine Matches gefunden!")
</VSCode.Cell>
<VSCode.Cell language="python">
# 📋 MATCH-DETAILS ANZEIGEN
if not matches_df.empty:
    print("📋 === MATCH-BEISPIELE ===")
    
    # Zeige beste Matches
    top_matches = matches_df.nlargest(5, 'confidence')
    
    for _, match in top_matches.iterrows():
        print(f"\n🎯 {match['match_type']} (Confidence: {match['confidence']:.3f})")
        print(f"   VDEH: '{match['vdeh_title']}'")
        print(f"   UB:   '{match['ub_title']}'")
        if match.get('vdeh_authors') and match.get('ub_authors'):
            print(f"   Autoren VDEH: {match['vdeh_authors']}")
            print(f"   Autoren UB:   {match['ub_authors']}")
    
    # Zeige Confidence-Verteilung
    print(f"\n📊 Confidence-Verteilung:")
    confidence_bins = pd.cut(matches_df['confidence'], bins=[0, 0.5, 0.75, 0.9, 0.95, 1.0])
    confidence_counts = confidence_bins.value_counts().sort_index()
    for bin_range, count in confidence_counts.items():
        print(f"   {bin_range}: {count:,} Matches")
</VSCode.Cell>
<VSCode.Cell language="python">
# 🕳️ LÜCKEN-ANALYSE
print("🕳️ === ERWERBUNGSLÜCKEN-ANALYSE ===")

gap_analysis = matcher.analyze_gaps(df_vdeh, matches_df)

print(f"📊 === LÜCKEN-STATISTIK ===")
print(f"Total VDEH Records: {gap_analysis['total_vdeh_records']:,}")
print(f"Gefundene Matches: {gap_analysis['matched_records']:,}")
print(f"Lücken (nicht im UB-Bestand): {gap_analysis['gap_records']:,}")
print(f"Lücken-Quote: {gap_analysis['gap_percentage']:.1f}%")

# Lücken nach Jahr
if gap_analysis.get('gaps_by_year'):
    print(f"\n📅 Lücken nach Jahr (Top 5):")
    for year, count in list(gap_analysis['gaps_by_year'].items())[:5]:
        print(f"   {year}: {count:,} Records")

# Lücken nach Sprache  
if gap_analysis.get('gaps_by_language'):
    print(f"\n🌍 Lücken nach Sprache:")
    for lang, count in gap_analysis['gaps_by_language'].items():
        print(f"   {lang}: {count:,} Records")
</VSCode.Cell>
<VSCode.Cell language="python">
# 💾 ERGEBNISSE EXPORTIEREN
print("💾 === ERGEBNISSE EXPORTIEREN ===")

comparison_dir = config.get_path('data.comparison.root')
reports_dir = config.get_path('data.comparison.reports')

# Match-Ergebnisse speichern
if not matches_df.empty:
    matches_path = comparison_dir / 'matches' / 'vdeh_ub_matches_sample.csv'
    matches_df.to_csv(matches_path, index=False, encoding='utf-8')
    print(f"✅ Matches gespeichert: {matches_path}")

# Statistiken speichern
stats = {
    'comparison_date': pd.Timestamp.now().isoformat(),
    'vdeh_sample_size': len(df_vdeh),
    'ub_sample_size': len(df_ub),
    'matcher_stats': matcher.get_statistics(),
    'gap_analysis': gap_analysis,
    'configuration_used': {
        'strategies': strategies,
        'thresholds': thresholds
    }
}

import json
stats_path = reports_dir / 'comparison_stats_sample.json'
with open(stats_path, 'w', encoding='utf-8') as f:
    json.dump(stats, f, indent=2, ensure_ascii=False, default=str)

print(f"✅ Statistiken gespeichert: {stats_path}")
print(f"\n🎯 Bestandsvergleich (Sample) abgeschlossen!")
print(f"📈 {len(matches_df) if not matches_df.empty else 0:,} Matches bei {len(df_vdeh):,} VDEH Records")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 📊 Visualisierung der Ergebnisse

Erstellt Grafiken für besseres Verständnis der Vergleichsergebnisse.
</VSCode.Cell>
<VSCode.Cell language="python">
# 📊 VISUALISIERUNG
if not matches_df.empty:
    plt.figure(figsize=(15, 10))
    
    # 1. Match-Strategien
    plt.subplot(2, 2, 1)
    strategy_counts = matches_df['match_type'].value_counts()
    plt.pie(strategy_counts.values, labels=strategy_counts.index, autopct='%1.1f%%')
    plt.title('Matches nach Strategie')
    
    # 2. Confidence-Verteilung
    plt.subplot(2, 2, 2)
    plt.hist(matches_df['confidence'], bins=20, alpha=0.7, edgecolor='black')
    plt.xlabel('Confidence Score')
    plt.ylabel('Anzahl Matches')
    plt.title('Confidence-Verteilung')
    
    # 3. Match-Rate Übersicht
    plt.subplot(2, 2, 3)
    categories = ['Matches', 'Keine Matches']
    values = [len(matches_df), len(df_vdeh) - len(matches_df)]
    plt.bar(categories, values, color=['green', 'orange'], alpha=0.7)
    plt.ylabel('Anzahl Records')
    plt.title(f'Match-Übersicht (n={len(df_vdeh):,})')
    
    # Werte auf Balken anzeigen
    for i, v in enumerate(values):
        plt.text(i, v + len(df_vdeh)*0.01, f'{v:,}', ha='center')
    
    # 4. Lücken nach Jahr (falls verfügbar)
    plt.subplot(2, 2, 4)
    if gap_analysis.get('gaps_by_year'):
        years = list(gap_analysis['gaps_by_year'].keys())[:5]
        counts = list(gap_analysis['gaps_by_year'].values())[:5]
        plt.bar(years, counts, alpha=0.7)
        plt.xlabel('Jahr')
        plt.ylabel('Anzahl Lücken')
        plt.title('Erwerbungslücken nach Jahr')
        plt.xticks(rotation=45)
    else:
        plt.text(0.5, 0.5, 'Keine Jahres-Daten\nverfügbar', ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('Jahres-Analyse')
    
    plt.tight_layout()
    
    # Speichere Grafik
    fig_path = reports_dir / 'comparison_overview_sample.png'
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    print(f"📊 Visualisierung gespeichert: {fig_path}")
    
    plt.show()
else:
    print("📊 Keine Daten für Visualisierung verfügbar (keine Matches gefunden)")
</VSCode.Cell>
```